In [ ]:
import os
import csv
import time
import folium
import random
import pulp 
import itertools
import numpy as np
import pandas as pd
from tqdm import tqdm
from geopy.distance import geodesic
from typing import List, Dict, Tuple, Generator
from sklearn.cluster import KMeans

In [ ]:
city_count= 50 #Nombre de villes de l'algorithme
truck_number = 6 #Nombre de camions pour la version avec camions
start_time = 480 #Heure de départ en minutes

population_size = 500 #Taille de la population de l'algo gén
num_generations = 500 #Nombre de génération
children_size = 498 #Nombre d'enfants issus des parents précédents
mutation_rate = 99 #taux de mutation
tournament_size = 20 #Taille du pool de sélection de parents
elite_size = 1 #Nombre d'élites issues de la génération précédente en plus du premier
num_iteration = 5 #Nombre d'itérations du multistart
initial_random_seed = 3

In [ ]:
def reset_to_default_seed():
    random.seed(initial_random_seed)
    np.random.seed(initial_random_seed)
    resetGlobal()

In [ ]:
def resetGlobal():
    global globalcity_count,truck_number,start_time,population_size,num_generations,children_size,mutation_rate,tournament_size,elite_size ,num_iteration
    city_count= 50 #Nombre de villes de l'algorithme
    truck_number = 6 #Nombre de camions pour la version avec camions
    start_time = 480 #Heure de départ en minutes

    population_size = 500 #Taille de la population de l'algo gén
    num_generations = 1600 #Nombre de génération
    children_size = 398 #Nombre d'enfants issus des parents précédents
    mutation_rate = 99 #taux de mutation
    tournament_size = 20 #Taille du pool de sélection de parents
    elite_size = 1 #Nombre d'élites issues de la génération précédente en plus du premier
    num_iteration = 5 #Nombre d'itérations du multistart

In [ ]:
####### Global performance def #######

"""

The purpose of this code is to define a decorator function called measure_time that 
can be used to measure the execution time of other functions. By applying this decorator 
to a function, you can easily track how long it takes to execute.

When the measure_time decorator is applied to a function, it wraps the original 
function with a wrapper function. The wrapper function checks if the measurement is enabled. 
If it is, the wrapper function records the start time, calls the original function, 
records the end time, calculates the execution time, and prints the results. 
Finally, it returns the result of the original function.


"""

def measure_time(func):
    # Get the enabled value outside the wrapper
    enabled = measure_time.enabled  

    def wrapper(*args, **kwargs):
        # ensure we're using the latest settings
        enabled = measure_time.enabled  
        if enabled:
            
            start_time = time.time()  
            # Call the original function with the provided arguments
            result = func(*args, **kwargs)  
            end_time = time.time()  
            execution_time = end_time - start_time 
            
            print(f" the Function: {func.__name__} took: {execution_time} seconds") 
            return result  # Return the result of the decorated function
        else:
            # If the decorator is disabled, simply call the original function
            return func(*args, **kwargs)  

    # Return the wrapper function only if enabled, else return the original function
    return wrapper if enabled else func


####### ####### ####### ####### #######

In [ ]:
####### Global ploting system #######

"""

The purpose of this code is to define a function called plot_cities that creates a map 
with markers and lines to visualize city data. The function takes 
two parameters: city_data, which contains the information about the cities to be 
plotted, and bound (optional), which specifies the order of cities to be plotted.

The plot_cities function contains several helper functions:

* arrange_cities_nearest: This function arranges the cities in city_data in the order 
specified by path. It sorts the cities based on their index in the path list, ensuring 
that the cities are plotted in the desired order.

* create_map_center: This function extracts the coordinates of the first city in 
city_data and returns them as the center of the map.

* create_custom_icon_style: This function defines a custom icon style for the markers 
on the map. It sets the background color, text color, border radius, padding, font weight, 
and font size.

* add_marker_with_icon: This function adds a marker with a custom icon to the map. 
It takes the coordinates, name, icon style, and index as parameters and creates a marker 
with a numbered icon at the specified coordinates.

* draw_line_between_cities: This function draws a line between two cities on the map. 
It takes the coordinates of the current city, the coordinates of the next city, the names 
of both cities, and the index as parameters. It creates a polyline connecting the two cities with 
a tooltip indicating the route number and the city names.

"""

color = ["Green", "Blue", "Yellow", "Orange", "Purple", "Pink", "Brown", "Gray", "Black", "White","Red"]

def plot_cities(city_data, bound=None, color='blue', map_obj=None):
    
    def arrange_cities_nearest(city_data, path):
        city_names = city_data[:, -1]
        sorted_indices = np.argsort([path.index(city) for city in city_names])
        sorted_city_data = city_data[sorted_indices]
        return np.vstack((sorted_city_data, sorted_city_data[0]))

    def create_map_center():
        return [float(city_data[0][2]), float(city_data[0][3])]

    def create_custom_icon_style():
        return """
            background-color: #ff5959;
            color: #ffffff;
            border-radius: 100%;
            padding: 20%;
            text-align: center;
            font-weight: bold;
            font-size: auto;
        """

    def add_marker_with_icon(coords, name, icon_style, i):
        folium.Marker(
            coords,
            popup=name,
            icon=folium.DivIcon(
                icon_size=(24, 24),
                icon_anchor=(12, 12),
                html='<div style="{}">{}</div>'.format(icon_style, i + 1)
            )
        ).add_to(map_obj)

    def draw_line_between_cities(coords, next_coords, name, next_city, i):
        folium.PolyLine(
            [coords, next_coords],
            color=color,
            weight=2.5,
            opacity=1.0,
            tooltip='Route {}: {} -> {}'.format(i + 1, name, next_city[4])
        ).add_to(map_obj)

    # Create a map if not provided
    if map_obj is None:
        map_center = create_map_center()
        map_obj = folium.Map(location=map_center, zoom_start=6)

    if bound:
        city_data = arrange_cities_nearest(city_data, bound)
        # Iterate over the city data and plot markers and lines
        for i in range(len(city_data) - 1):
            city = city_data[i]
            coords = [float(city[2]), float(city[3])]
            name = city[4]

            icon_style = create_custom_icon_style()

            add_marker_with_icon(coords, name, icon_style, i)

            next_city = city_data[i + 1]
            next_coords = [float(next_city[2]), float(next_city[3])]

            draw_line_between_cities(coords, next_coords, name, next_city, i)
    else:
        # Iterate over the city data and create a folium marker for each city
        for city in city_data:
            folium.Marker(
                location=[float(city[2]), float(city[3])],
                popup=city[4]
            ).add_to(map_obj)

    # Display the map
    return map_obj

####### ####### ####### ####### #######

In [ ]:
####### Global notebook configs #######

# Toggle for enabling/disabling the 
# decorator
measure_time.enabled = True 
if measure_time.enabled:
    print("* measure_time is enabled ")

# specify the folder path and files name 
dataset_file_path = os.path.join('../datasets', 'cities.csv')
print(f"* the selected dataset is located at: {dataset_file_path}")

####### ####### ####### ####### #######

# City Generator 

this part contains the folowing logic: we first retrieve data from a dataset and later construct a sample from it that contain the cities name, the ZIP Code, the population count and longitude|latitude 

<u>non-optimized</u>

In [ ]:
@measure_time
def read_csv_to_tuple(filename: str):
    with open(filename, "r", encoding='ISO-8859-1') as fh:  # Open the file in read mode
        # Create a CSV reader object with delimiter ';'
        reader = csv.reader(fh, delimiter=';')  
        # Skip the header row
        next(reader, None)  
        # Convert the remaining rows to a tuple
        cities = tuple(reader)  
    return cities  # Return the tuple


@measure_time
def sample_N_from_tuple(cities: tuple, size: int = None):
    totalRows = len(cities)
    # If size is not specified or greater than totalRows Return 
    # an empty tuple
    if size is None or size > totalRows:  
        return ()
    # Return a random sample of 'size' elements from the tuple
    return random.sample(cities, size)  

In [ ]:
citiesTuple = read_csv_to_tuple(dataset_file_path)
citiesSample = sample_N_from_tuple(citiesTuple, city_count)

<u>optimized</u>

- Reads the CSV file using the pandas library's read_csv function.
    - Efficient CSV reading: The optimized version uses pandas' read_csv function, which is highly optimized for reading CSV files. It takes advantage of optimized file parsing algorithms and efficient memory management, resulting in faster file reading compared to the line-by-line reading in the non-optimized version.

- Converts the DataFrame to a tuple of lists and then to a tuple.
    - In the non-optimized version, each row from the CSV file is converted to a tuple individually. In the optimized version, pandas converts the entire DataFrame to a tuple of lists in one operation, which is more efficient and faster.
    
- Uses pandas and NumPy functions for sampling instead of the random.sample function.
    - The non-optimized version uses the random.sample function to sample elements from the tuple. In the optimized version, NumPy's np.random.choice function is used, which is implemented in optimized C code and performs faster random sampling.
    - The optimized version uses pandas' iloc function to extract the sampled data based on the selected indices. This indexing operation is optimized in pandas and provides faster access to the desired rows.

In [ ]:
@measure_time
def read_csv_to_tuple(filename: str):
    # Read the CSV file using pandas
    df = pd.read_csv(filename, delimiter=';', encoding='ISO-8859-1')
    # Convert the DataFrame to a tuple of lists and then to a tuple
    cities = tuple(df.values.tolist())  
    return cities


@measure_time
def sample_N_from_tuple(cities: tuple, size: int = None):
    # Create a DataFrame from the tuple of lists
    df = pd.DataFrame(list(cities))
    # Get the total number of rows in the DataFrame
    totalRows = len(df)
    
    # If size is not specified or greater than totalRows
    # Return an empty tuple
    if size is None or size > totalRows:  
        return ()
    
    # Randomly select 'size' indices without replacement
    indices = np.random.choice(totalRows, size, replace=False)
    # Extract the sampled data based on the selected indices
    sampled_data = df.iloc[indices].values.tolist()  
    # Return the sampled data as a NumPy array
    return np.array(sampled_data)  

In [ ]:
reset_to_default_seed()

citiesTuple = read_csv_to_tuple(dataset_file_path)
citiesSample = sample_N_from_tuple(citiesTuple, city_count)

<u>the actual output of the city generator section</u>

In [ ]:
# disable performance profiling for this section 
measure_time.enabled = False
reset_to_default_seed()

citiesTuple = read_csv_to_tuple(dataset_file_path)
citiesSample = sample_N_from_tuple(citiesTuple, city_count)

# display the map with the selected cities
plot_cities(citiesSample)

# location generator 
The purpose of this staged is to generate a series of city names along with their respective longitude and latitude coordinates. It achieves this by extracting the relevant information from a given list of city data

In [ ]:
# enable performance profiling for this section 
measure_time.enabled = True

<u>non-optimized</u>

In [ ]:
@measure_time
def create_location_generator(citiesSample: List[List[str]]) -> Dict[str, Tuple[float, float]]:
    # Create an empty dictionary to store the location data
    tmp = {}  
    
    for city in citiesSample:  # Iterate over each city in citiesSample
        city_name = city[4]  # Get the city name from the city data
        longitude = float(city[3])  # Get the longitude from the city data and convert it to float
        latitude = float(city[2])  # Get the latitude from the city data and convert it to float
        population = int(city[1]) # Get the population from the city data and convert it to int
        tmp[city_name] = (longitude, latitude,population)  # Store the population, longitude and latitude as a tuple in the dictionary
    
    # Return the dictionary containing the location data
    return tmp

In [ ]:
location = create_location_generator(citiesSample)

<u>optimized</u>

- Uses NumPy array indexing to extract city names, longitudes, and latitudes from the citiesSample list in one operation.
    - The optimized version leverages NumPy's array indexing and vectorized operations to extract the necessary data from the citiesSample list. This allows for faster and more efficient data extraction compared to the iterative approach in the non-optimized version.
- Converts the longitudes and latitudes to float using NumPy's astype function.
    - In the non-optimized version, the conversion to float is performed individually for each longitude and latitude. In the optimized version, NumPy's astype function is applied to the entire arrays of longitudes and latitudes in one operation. This bulk conversion is more efficient and faster.
- Utilizes the zip function and generator syntax (yield from) to create a generator that yields tuples of city names and corresponding longitude-latitude pairs.
    - The optimized version uses a generator and the yield from syntax to produce the desired output. Generators provide a memory-efficient way to produce values on-the-fly, as opposed to constructing and returning a complete dictionary in the non-optimized version. This can improve performance, especially when dealing with large datasets.


In [ ]:
@measure_time
def create_location_generator(citiesSample: List[List[str]]) -> Generator[Tuple[str, Tuple[float, float]], None, None]:
    # Extract city names from citiesSample using NumPy array indexing
    city_names = np.array(citiesSample)[:, 4]
    # Extract longitudes and convert them to float using NumPy array indexing
    longitudes = np.array(citiesSample)[:, 3].astype(float)
    # Extract latitudes and convert them to float using NumPy array indexing
    latitudes = np.array(citiesSample)[:, 2].astype(float)
    # Extract population and convert them to int using NumPy array indexing
    population = np.array(citiesSample)[:, 1].astype(int)

    yield from zip(city_names, zip(longitudes, latitudes,population))

In [ ]:
location = create_location_generator(citiesSample)

<u>the actual output of the location generator section</u>

In [ ]:
# disable performance profiling for this section 
measure_time.enabled = False

for city_name, coordinates in create_location_generator(citiesSample):
    print(f'{city_name}: {coordinates}')

# distance matrix generator

this part calculate a distance matrix for a set of cities based on their geographic coordinates. 

In [ ]:
# enable performance profiling for this section 
measure_time.enabled = True

<u>non-optimized</u>

In [ ]:
@measure_time
def calculate_distance_matrix(generator) -> Dict[str, Dict[str, float]]:
    distance_matrix = {}  # Create an empty dictionary to store the distance matrix
    city_coords = []  # Create an empty list to store city names and coordinates
    
    # Iterate over each city name and coordinates from the generator and 
    # append the city name and coordinates as a tuple to city_coords
    for city_name, coordinates in generator:
        city_coords.append((city_name, tuple([coordinates[1],coordinates[0],0]))) 
    # Iterate over the city name and coordinates using enumerate
    for i, (city1, coords1) in enumerate(city_coords):
        
        # Create an empty dictionary for each city in the distance matrix
        distance_matrix[city1] = {}
        
        # Iterate over the city name and coordinates again
        for j, (city2, coords2) in enumerate(city_coords):  
            if i == j:
                # Set the distance between a city and itself to 0.0
                distance_matrix[city1][city2] = 0.0 
            else:
                # Calculate the geodesic distance between two coordinates
                distance = geodesic(coords1, coords2).kilometers
                # Store the distance in the distance matrix
                distance_matrix[city1][city2] = distance / 51.3
    
    return distance_matrix

In [ ]:
distance_matrix = calculate_distance_matrix(
    create_location_generator(citiesSample)
)

<u>optimized</u>

- The optimized version directly stores the city coordinates in a dictionary (city_coords), eliminating the need for additional data structures like the city_coords list in the non-optimized version. This reduces memory usage and unnecessary operations, resulting in improved performance.
- The optimized version utilizes NumPy's vectorized operations to calculate distances between pairs of coordinates. By converting the coordinates to a NumPy array and using broadcasting, the calculations can be performed efficiently in parallel, leading to significant speed improvements.
- Instead of constructing an empty dictionary for each city, the optimized version creates a square matrix (distances) with zeros to store the distance values. This allows for efficient indexing and updating of the distances using NumPy operations.
- The optimized version converts the distances matrix to a pandas DataFrame, which provides efficient indexing capabilities and convenient conversion to a dictionary. This avoids nested loops and dictionary updates in the non-optimized version, resulting in improved performance.

<u>the actual output of the distance matrix generator section</u>

In [ ]:
# disable performance profiling for this section 
measure_time.enabled = False

calculate_distance_matrix(
    create_location_generator(citiesSample)
)

# distance matrix generator with frequency

## <u>non-optimized</u>

L'objectif est de générer une matrice de distance prenant en compte la fréquentation. <br> Ainsi nous présenterons la fréquentation sous la forme d'un tableau contenant 4 valeurs (temps de trajets) indiqués à différents moments de la journée (les deux moitiés matin et après midi).


In [ ]:
@measure_time
def generate_distances_frequency(distances):
    distances_frequencies = {}

    for city_a, times in distances.items():
        distances_frequencies[city_a] = {}

        for city_b, time in times.items():
            if city_a == city_b:
                distances_frequencies[city_a][city_b] = [0 for i in range(4)]  # Assuming no traffic within the same city
            else:
                distances_frequencies[city_a][city_b] = [np.random.uniform(1.0, 1.5) * time for i in range(4)]
    
    return distances_frequencies

In [ ]:
generator = create_location_generator(citiesSample)
generate_distances_frequency(calculate_distance_matrix(generator))

## <u>optimized</u>

In [ ]:
print("TODO")

# Initial solutions provider (no constraints)

## <u>non-optimized</u>

In [ ]:
def generate_random_initial_solutions(distances,start_point):
    points = list(distances.keys())
    points.remove(start_point)
    remaining_points = points[:]
    path = [start_point]
    while remaining_points:
        next_point = np.random.choice(remaining_points)
        path.append(next_point)
        remaining_points.remove(next_point)
    return path

## <u>optimized</u>

In [ ]:
print("TODO")

In [ ]:
## <u>output</u>

In [ ]:
reset_to_default_seed()

generator = create_location_generator(citiesSample)
distances_km = calculate_distance_matrix(generator)

start = next(iter(distances_km))

path = generate_random_initial_solutions(distances_km,start)
print(path)

# genetic algorithm with frequency constraint

In [ ]:
# enable performance profiling for this section 
measure_time.enabled = True

<u>non-optimized</u>

In [ ]:
### 
def fitness(tour,distances,start_time): 
    current_time= start_time
    current_distance = 0
    for x in range(len(tour)-1):
        if current_time > 1440:
            current_time-=1440
        current_distance += distances[tour[x]][tour[x+1]][current_time//360]
    return current_distance

# Sélection des parents par tournoi
def selection_tournament(population,distances,start_time,tournament_size):
    selected_parents = []
    for _ in range(len(population)):
        participants = random.sample(population, tournament_size)
        winner = min(participants, key=lambda k: fitness(k, distances,start_time))
        selected_parents.append(winner)
    return selected_parents

# Croisement de chad
def crossover(parent1, parent2):
    div = 3
    liste_manquante = []
    offspring = []
    c1_part1 = parent1[0:int((len(parent1)) / div)]
    c1_part2 = parent1[int((len(parent1)) / div):int(len(parent1) - int((len(parent1)) / div))]
    c1_part3 = parent1[int(len(parent1) - int((len(parent1)) / div)):]
    c2_part1 = parent2[0:int((len(parent2)) / div)]
    c2_part2 = parent2[int((len(parent2)) / div):int(len(parent2) - int((len(parent2)) / div))]
    c2_part3 = parent2[int(len(parent2) - int((len(parent2)) / div)):]
    for i in range(0, len(c2_part3)):
        if c2_part3[i] not in c1_part2:
            liste_manquante.append(c2_part3[i])
    for i in range(0, len(c2_part1)):
        if c2_part1[i] not in c1_part2:
            liste_manquante.append(c2_part1[i])
    for i in range(0, len(c2_part2)):
        if c2_part2[i] not in c1_part2:
            liste_manquante.append(c2_part2[i])
    for x in range(0, len(c1_part1)):
        offspring.append(liste_manquante[x - len(c1_part3)])
    offspring.extend(c1_part2)
    for x in range(0, len(c1_part3)):
        offspring.append(liste_manquante[x])
    return offspring;

# Opération 2-opt
def two_opt(tour,distances_freq,start_time): #O(n^2)
    improved = True
    best_distance = fitness(tour,distances_freq,start_time)

    while improved:
        improved = False
        for i in range(1, len(tour) - 1):
            for j in range(i + 1, len(tour)):
                new_tour = tour[:i] + tour[i:j][::-1] + tour[j:]
                new_distance = fitness(new_tour,distances_freq,start_time)
                if new_distance < best_distance:
                    tour = new_tour
                    best_distance = new_distance
                    improved = True

    return tour

def mutate(tour,mutation_rate):
    p = np.random.randint(0, 100)
    if p < mutation_rate:
        gene1 = np.random.randint(1, len(tour)-2)
        gene2 = np.random.randint(1, len(tour)-2)
        tour[gene1], tour[gene2] = tour[gene2], tour[gene1]
    return tour;

def select_parents(population, distances_freq,starttime,tournament_size):
    fitness_sum = sum(fitness(individual,distances_freq,starttime) for individual in population)
    probabilities = [fitness(individual,distances_freq,starttime) / fitness_sum for individual in population]
    parents = random.choices(population, weights=probabilities, k=tournament_size)
    return parents

def fill_pop_with_random(population,range_for,start_point,distances):
    for i in range(range_for):
        population.append(generate_random_initial_solutions(distances, start_point))
    return population

# Algorithme génétique
@measure_time
def genetic_algorithm(distances_freq,population_size,num_generations,mutation_rate,tournament_size,start_point,starttime,children_size,elite_size):
        
    best_distance = 0 # O(1)
    optimum_count = 0 # O(1)
    
    progress_bar = tqdm( # O(1)
        total=num_generations, 
        desc="Genetic Algorithm", 
        unit="generation", 
    )
    
    # Initialisation de la population
    population = fill_pop_with_random([],population_size,start_point,distances_freq)# O(n)
    for gen in range(num_generations):
        # Sélection des parents
        parents = selection_tournament(population,distances_freq,starttime,tournament_size)# O(n * n)

        parents = sorted(parents, key=lambda k: fitness(k, distances_freq,starttime))# O(n * log(n))
        
        #Ajout d'une élite à la population
        population = [parents[0]] + random.sample(parents[:tournament_size],elite_size) # O(n)
        offspring = []
        for i in range(children_size):
            parent1, parent2 = random.choice(parents[:tournament_size]),parents[i]# O(1)
            child = crossover(parent1, parent2)# O(len(n))
            child = mutate(child,mutation_rate)# O(len(n))
            offspring.append(child)# O(1)
            
        # Remplacement de la population
        population = population + offspring # O(len(n))
        population = fill_pop_with_random(population,population_size-len(population),start_point,distances_freq)# O(n - len(n))
            
        currentwinner = min(population, key=lambda k: fitness(k, distances_freq,starttime))# O(n)
        
        best_distance = fitness(currentwinner, distances_freq,starttime)# O(len(n))
        #print("Current winner with : ",best_distance)
        #print("Current pop count : ",len(population))
        #print("Current generation : ",gen)
        #print("---------")
        progress_bar.update(1)
        

    # Meilleure tournée après les générations
    best_tour = min(population, key=lambda k: fitness(k, distances_freq,starttime))# O(n)
    best_tour.append(best_tour[0])# O(1)
    
    #Optimisation finale
    best_tour = two_opt(best_tour,distances_freq,starttime)#O(n)
    
    # Évaluation de la meilleure tournée
    best_distance = fitness(best_tour,distances_freq,starttime)#O(len(n))

    return best_tour, best_distance

In [ ]:
generator = create_location_generator(citiesSample)
distances_freq = generate_distances_frequency(calculate_distance_matrix(generator))

start_point = next(iter(distances_freq.keys())) # this represent the first town found in the distances dict 
# this represent the first town found in the distances dict 

path = genetic_algorithm(distances_freq,population_size,num_generations,mutation_rate,tournament_size,start_point,start_time,children_size,elite_size)

<u>optimized</u>

- The optimized version utilizes NumPy's vectorized operations to perform calculations on arrays of distances and boolean masks. These operations can be executed efficiently in parallel, leading to improved performance compared to the iterative approach in the non-optimized version.
- The optimized version reduces the number of conditional checks by updating the nearest town and minimum distance within the vectorized operations. This avoids multiple checks and updates within the loop, resulting in faster execution.
- The optimized version uses NumPy's array indexing functions (isin and argmin) to identify unvisited towns and find the index of the nearest town with the shortest distance. These indexing operations are optimized in NumPy and provide faster access and retrieval of elements from arrays.


In [ ]:
print("TODO")

In [ ]:
generator = create_location_generator(citiesSample)
distances_freq = generate_distances_frequency(calculate_distance_matrix(generator))

start_point = next(iter(distances_freq.keys())) # this represent the first town found in the distances dict 
# this represent the first town found in the distances dict 

path = genetic_algorithm(distances_freq,population_size,num_generations,mutation_rate,tournament_size,start_point,start_time,children_size,elite_size)

<u>the actual output of the genetic algorithm</u>

In [ ]:
generator = create_location_generator(citiesSample)
distances_freq = generate_distances_frequency(calculate_distance_matrix(generator))

start_point = next(iter(distances_freq.keys())) # this represent the first town found in the distances dict 
# this represent the first town found in the distances dict 

path = genetic_algorithm(distances_freq,population_size,num_generations,mutation_rate,tournament_size,start_point,start_time,children_size,elite_size)
print("Path:", ' -> '.join(path[0]))
print("Time to travel :",fitness(path[0],distances_freq,start_time))

In [ ]:
plot_cities(citiesSample, path[0])

# genetic algorithm with all constraints

In [ ]:
def split_into_trucks(time_matrix, num_trucks):
    # Convert the time matrix to a numpy array
    towns = list(time_matrix.keys())
    time_array = np.array([[np.average(time_matrix[town1][town2]) for town2 in towns] for town1 in towns])
    
    
    # Apply k-means clustering
    kmeans = KMeans(n_clusters=num_trucks, random_state=0, n_init=10).fit(time_array)
    labels = kmeans.labels_
    
    # Find the index of the first town in the time matrix
    first_town_index = towns.index(towns[0])
    
    
    # Create time matrices for each truck
    trucks = []
    for i in range(num_trucks):
        # Filter towns based on the label or if it's the first town
        truck_towns = [towns[j] for j in range(len(towns)) if labels[j] == i or j == first_town_index]
        truck_time_matrix = {town: {truck_town: time_matrix[town][truck_town] for truck_town in truck_towns} for town in truck_towns}
        trucks.append(truck_time_matrix)
    
    return trucks


def fitness(tour,distances,start_time):
    current_time= start_time
    current_distance = 0
    for x in range(len(tour)-1):
        if current_time > 1440:
            current_time-=1440
        current_distance += distances[tour[x]][tour[x+1]][current_time//360]
    return current_distance

# Sélection des parents par tournoi
def selection_tournament(population,distances,start_time,tournament_size):
    selected_parents = []
    for _ in range(len(population)):
        participants = random.sample(population, tournament_size)
        winner = min(participants, key=lambda k: fitness(k, distances,start_time))
        selected_parents.append(winner)
    return selected_parents

# Croisement
def crossover(parent1, parent2):
    div = 3
    offspring = [parent1[0]]  # Le premier élément de parent1 est ajouté à offspring
    c1_part1 = parent1[0:int((len(parent1)) / div)]
    c1_part2 = parent1[int((len(parent1)) / div):int(len(parent1) - int((len(parent1)) / div))]
    c1_part3 = parent1[int(len(parent1) - int((len(parent1)) / div)):]
    c2_part1 = parent2[0:int((len(parent2)) / div)]
    c2_part2 = parent2[int((len(parent2)) / div):int(len(parent2) - int((len(parent2)) / div))]
    c2_part3 = parent2[int(len(parent2) - int((len(parent2)) / div)):]
    
    for i in range(0, len(c2_part3)):
        if c2_part3[i] not in c1_part2:
            offspring.append(c2_part3[i])
    for i in range(0, len(c2_part1)):
        if c2_part1[i] not in c1_part2 and c2_part1[i] not in offspring:
            offspring.append(c2_part1[i])
    for i in range(0, len(c2_part2)):
        if c2_part2[i] not in c1_part2 and c2_part2[i] not in offspring:
            offspring.append(c2_part2[i])
    
    offspring.extend(c1_part2)
    
    for x in range(0, len(c1_part3)):
        if c1_part3[x] not in offspring:
            offspring.append(c1_part3[x])
            
    return offspring


# Opération 2-opt
def two_opt(tour,distances_freq,start_time):
    improved = True
    best_distance = fitness(tour,distances_freq,start_time)

    while improved:
        improved = False
        for i in range(1, len(tour) - 1):
            for j in range(i + 1, len(tour)):
                new_tour = tour[:i] + tour[i:j][::-1] + tour[j:]
                new_distance = fitness(new_tour,distances_freq,start_time)
                if new_distance < best_distance:
                    tour = new_tour
                    best_distance = new_distance
                    improved = True

    return tour

def mutate(tour,mutation_rate):
    p = random.randint(0, 100)
    if p < mutation_rate:
        gene1 = random.randint(1, len(tour)-2)
        gene2 = random.randint(1, len(tour)-2)
        tour[gene1], tour[gene2] = tour[gene2], tour[gene1]
    return tour;

def select_parents(population, distances_freq,starttime,tournament_size):
    fitness_sum = sum(fitness(individual,distances_freq,starttime) for individual in population)
    probabilities = [fitness(individual,distances_freq,starttime) / fitness_sum for individual in population]
    parents = random.choices(population, weights=probabilities, k=tournament_size)
    return parents

def fill_pop_with_random(population,range_for,start_point,distances):
    for i in range(range_for):
        population.append(generate_random_initial_solutions(distances, start_point))
    return population

def filter_cities_by_truck(citiesSample, truck):
    town_list = list(truck.keys())
    filtered_cities = [city for city in citiesSample if city[4] in town_list]
    return np.array(filtered_cities)

# Algorithme génétique
@measure_time
def genetic_algorithm(distances_freq,population_size,num_generations,mutation_rate,tournament_size,start_point,starttime,children_size,elite_size):
    
    best_distance = 0  # O(1)
    optimum_count = 0   # O(1)
    
    progress_bar = tqdm(
        total=num_generations, 
        desc="Genetic Algorithm", 
        unit="generation", 
    )
    # Initialisation de la population
    population = fill_pop_with_random([],population_size,start_point,distances_freq)  # O(n)
    for gen in range(num_generations):
        # Sélection des parents
        parents = selection_tournament(population,distances_freq,starttime,tournament_size)  # O(n*n)

        parents = sorted(parents, key=lambda k: fitness(k, distances_freq,starttime))  # O(n * log(n))
        
        #Ajout d'une élite à la population
        population = [parents[0]] + random.sample(parents[:tournament_size],elite_size)  # O(n)
        offspring = []
        for i in range(children_size):
            parent1, parent2 = random.choice(parents[:tournament_size]),parents[i]  # O(1)
            child = crossover(parent1, parent2)  # O(len(n))
            child = mutate(child,mutation_rate)  # O(len(n))
            offspring.append(child)  # O(1)
        # Remplacement de la population
        population = population + offspring
        population = fill_pop_with_random(population,population_size-len(population),start_point,distances_freq)
            
        currentwinner = min(population, key=lambda k: fitness(k, distances_freq,starttime))
        
        best_distance = fitness(currentwinner, distances_freq,starttime)
        #print("Current winner with : ",best_distance)
        #print("Current pop count : ",len(population))
        #print("Current generation : ",gen)
        #print("---------")
        progress_bar.update(1)
        

    # Meilleure tournée après les générations
    best_tour = min(population, key=lambda k: fitness(k, distances_freq,starttime))  # O(n)
    best_tour.append(best_tour[0])  # O(1)
    
    #Optimisation finale
    best_tour = two_opt(best_tour,distances_freq,starttime)# O(n^2)
    
    # Évaluation de la meilleure tournée
    best_distance = fitness(best_tour,distances_freq,starttime)# O(len(n))


    return best_tour, best_distance

def optimize_trucks(map_obj,citiesSample,trucks,population_size,num_generations,mutation_rate,tournament_size,start_point,start_time,children_size,elite_size, use_traffic_matrix=False):
    routes = []
    for i, truck in enumerate(trucks):
        if use_traffic_matrix:
            truck = generate_traffic_matrix(truck)
        best_route, best_time = genetic_algorithm(truck,population_size,num_generations,mutation_rate,tournament_size,start_point,start_time,children_size,elite_size)
        
        print(f"The truck n: {i} will do: {best_route} for a total of {best_time} h")
        plot_cities(filter_cities_by_truck(citiesSample, truck), best_route, map_obj=map_obj, color=color[i])
        routes.append([best_route, best_time])
    return map_obj, routes

In [ ]:
map_center = [float(citiesSample[0][2]), float(citiesSample[0][3])]
map_obj = folium.Map(location=map_center, zoom_start=6)


generator = create_location_generator(citiesSample)
distances_km = calculate_distance_matrix(generator)
distances_freqU = generate_distances_frequency(distances_km)
trucks = split_into_trucks(distances_freqU,truck_number)

start_point = next(iter(distances_km.keys())) # this represent the first town found in the distances dict 
# this represent the first town found in the distances dict 

print(start_point)

map_obj,routes = optimize_trucks(map_obj,citiesSample,trucks,population_size,num_generations,mutation_rate,tournament_size,start_point,start_time,children_size,elite_size)

map_obj

# Upper bound

In [ ]:
np.random.seed()

In [ ]:
@measure_time
def solve_issue(trucks):
    max_value = 0
    # solve with pulp
    for initial_truck in trucks:
        truck = {}
        for element in initial_truck:
            truck[element] = {}
            for subelement in initial_truck[element]:
                truck[element][subelement] = np.round(initial_truck[element][subelement])
            
        # definition of LpProblem instance
        problem =pulp.LpProblem("CVRP", pulp.LpMinimize)

        truck_keys = list(truck.keys())
        
        # definition of variables 
        x = [[[pulp.LpVariable("x%s_%s"%(i,j), cat="Binary") if i != j else None] for j in truck_keys] for i in truck_keys]
        print(x)
        
        # add objective function
        problem += pulp.lpSum(np.min(truck[ikey][jkey]).astype(int) * x[iindex][jindex] if iindex != jindex else 0
                              for jindex,jkey in enumerate(truck_keys)
                              for iindex,ikey in enumerate(truck_keys))

        
        # constraints
        for j in range(1, len(truck)):
            problem += pulp.lpSum(x[iindex][j] if iindex != j else 0 
                                  for iindex,ikey in enumerate(truck_keys)) == 1 
        
        problem += pulp.lpSum(x[0][iindex] for iindex,ikey in enumerate(truck_keys)) == 1
        problem += pulp.lpSum(x[jindex][0] for jindex,jkey in enumerate(truck_keys)) == 1
        
        subtours = []
        for i in range(2,len(truck)):
             subtours += itertools.combinations(range(1,len(truck)), i)

        for s in subtours:
            problem += pulp.lpSum(x[i][j] if i !=j else 0 for i, j in itertools.permutations(s,2)) <= len(s) - 1

        
        problem.solve()
        print(pulp.value(problem.objective))
        if pulp.LpStatus[problem.status] == "Optimal":
            if(max_value < pulp.value(problem.objective)):
                max_value = pulp.value(problem.objective)
                print("NEW STRONK")
        else:
            raise Exception("No optimal solution found.")

    return max_value
        


In [ ]:
generator = create_location_generator(citiesSample)
distances_km = calculate_distance_matrix(generator)
distances_freq = generate_distances_frequency(distances_km)
trucks = split_into_trucks(distances_freq,truck_number)

In [ ]:
solve_issue(trucks)

# genetic algorithm with all constraints and multistart

In [ ]:
def multistart(num_iteration,citiesSample,trucks,population_size,num_generations,mutation_rate,tournament_size,start_point,start_time,children_size,elite_size):
    current_best_distance = float('inf')
    current_best_path = None
    for i in range(num_iteration):
        map_center = [float(citiesSample[0][2]), float(citiesSample[0][3])]
        map_obj = folium.Map(location=map_center, zoom_start=6)
        map_obj, paths = optimize_trucks(map_obj,citiesSample,trucks,population_size,num_generations,mutation_rate,tournament_size,start_point,start_time,children_size,elite_size)
        
        second_column = [sublist[1] for sublist in paths]
        max_value = np.max(np.array(second_column))
        
        current_path_distance = max_value
        
        if current_best_distance > current_path_distance:
            current_best_distance = current_path_distance
            current_best_path = map_obj
    return current_best_distance, current_best_path

In [ ]:
map_center = [float(citiesSample[0][2]), float(citiesSample[0][3])]
map_obj = folium.Map(location=map_center, zoom_start=6)

generator = create_location_generator(citiesSample)
distances_km = calculate_distance_matrix(generator)
distances_freq = generate_distances_frequency(distances_km)
trucks = split_into_trucks(distances_freq,truck_number)

start_point = next(iter(distances_km.keys())) # this represent the first town found in the distances dict 
# this represent the first town found in the distances dict 

print(start_point)

size,map_obj = multistart(num_iteration,citiesSample,trucks,population_size,num_generations,mutation_rate,tournament_size,start_point,start_time,children_size,elite_size)

print("With a maximum individual path size of :",size)
map_obj